## Installation de GeneticSharp


In [21]:
#r "nuget: GeneticSharp"

## Classe Portfolio

Cette classe Portfolio est utilisée pour représenter un portefeuille d'actifs financiers. Chaque actif a un prix et un rendement attendu. Cette classe inclut des méthodes pour calculer le retour total et le risque du portefeuille en fonction des poids des actifs.

In [22]:
using System;
using System.Collections.Generic;
using System.Linq;

// Classe Portfolio pour représenter un portefeuille d'actifs financiers
public class Portfolio
{
    // Liste des noms d'actifs
    public List<string> Assets { get; set; }
    
    // Dictionnaire avec les prix des actifs
    public Dictionary<string, double> Prices { get; set; }
    
    // Dictionnaire contenant les rendements attendus des actifs
    public Dictionary<string, double> ExpectedReturns { get; set; }

    // Constructeur pour initialiser le portefeuille chaque actif a un prix et un rendemenent associé 
    public Portfolio(List<string> assets, Dictionary<string, double> prices, Dictionary<string, double> expectedReturns)
    {
        Assets = assets;
        Prices = prices;
        ExpectedReturns = expectedReturns;
    }

    // Calcul du retour total du portefeuille
    public double CalculateReturn(Dictionary<string, double> weights)
    {
        double portfolioReturn = 0;
        foreach (var asset in Assets)
        {
            portfolioReturn += weights[asset] * ExpectedReturns[asset];
        }
        return portfolioReturn;
    }

    // Calcul du risque du portefeuille (simplifié pour l'instant)
    public double CalculateRisk(Dictionary<string, double> weights)
    {
        double risk = 0;
        foreach (var weight in weights.Values)
        {
            risk += Math.Pow(weight, 2);
        }
        return Math.Sqrt(risk); 
    }
}



## Définition des Chromosomes et de la Fonction de Fitness

La classe PortfolioChromosome représente un chromosome dans notre algorithme génétique. Chaque gène représente le poids d'un actif dans le portefeuille. Les poids sont initialisés aléatoirement et normalisés pour s'assurer que la somme des poids est égale à 1.

In [23]:
using GeneticSharp;
using System.Collections.Generic;
using System.Linq;

// Classe pour représenter un chromosome de portefeuille
public class PortfolioChromosome : ChromosomeBase
{
    private List<string> _assets;

    // Constructeur pour initialiser les actifs du chromosome
    public PortfolioChromosome(List<string> assets) : base(assets.Count)
    {
        _assets = assets;
        for (int i = 0; i < assets.Count; i++)
        {
            ReplaceGene(i, new Gene(RandomizationProvider.Current.GetDouble(0, 1)));
        }
    }

    // Génère un gène avec une valeur aléatoire
    public override Gene GenerateGene(int geneIndex)
    {
        return new Gene(RandomizationProvider.Current.GetDouble(0, 1));
    }

    // Crée un nouveau chromosome
    public override IChromosome CreateNew()
    {
        return new PortfolioChromosome(_assets);
    }

    // Obtient les poids des actifs représentés par le chromosome
    public Dictionary<string, double> GetWeights()
    {
        var weights = new Dictionary<string, double>();
        for (int i = 0; i < Length; i++)
        {
            weights.Add(_assets[i], (double)GetGene(i).Value);
        }
        NormalizeWeights(weights);
        return weights;
    }

    // Normalise les poids pour que leur somme soit égale à 1
    private void NormalizeWeights(Dictionary<string, double> weights)
    {
        double sum = weights.Values.Sum();
        var keys = weights.Keys.ToList();
        foreach (var key in keys)
        {
            weights[key] /= sum;
        }
    }
}


## Implémentation du Solveur par Algorithme Génétique

L'implémentation du solveur par algorithme génétique se fait en plusieurs étapes, incluant la définition des classes nécessaires, la configuration de l'algorithme génétique et l'exécution de l'algorithme pour trouver la meilleure solution possible.

In [24]:
using GeneticSharp;

// Classe pour évaluer la fitness d'un portefeuille
public class PortfolioFitness : IFitness
{
    private Portfolio _portfolio;

    // Constructeur pour initialiser le portefeuille
    public PortfolioFitness(Portfolio portfolio)
    {
        _portfolio = portfolio;
    }

    // Fonction d'évaluation de la fitness
    public double Evaluate(IChromosome chromosome)
    {
        var portfolioChromosome = chromosome as PortfolioChromosome;
        var weights = portfolioChromosome.GetWeights();
        double portfolioReturn = _portfolio.CalculateReturn(weights);
        double risk = _portfolio.CalculateRisk(weights);
        
        // Objectif : Maximiser le retour et minimiser le risque
        return portfolioReturn - risk;
    }
}


## Test du Solveur

In [25]:
//on définit une liste d'actifs financiers (assets), avec leurs prix (prices) et leurs rendements attendus (expectedReturns)
var assets = new List<string> { "Asset0", "Asset1", "Asset2", "Asset3", "Asset4" };
var prices = new Dictionary<string, double>
{
    { "Asset0", 100 },
    { "Asset1", 200 },
    { "Asset2", 300 },
    { "Asset3", 400 },
    { "Asset4", 500 }
};
var expectedReturns = new Dictionary<string, double>
{
    { "Asset0", 0.05 },
    { "Asset1", 0.10 },
    { "Asset2", 0.15 },
    { "Asset3", 0.20 },
    { "Asset4", 0.25 }
};

// Crée un portefeuille
var portfolio = new Portfolio(assets, prices, expectedReturns);

// Crée la fitness,Un objet PortfolioFitness est créé pour évaluer la "qualité" des solutions potentielles (portefeuilles) générées par l'algorithme génétique. 
//La méthode Evaluate de cette classe calcule le retour total et le risque du portefeuille, et essaie de maximiser le retour tout en minimisant le risque
var fitness = new PortfolioFitness(portfolio);

// Crée le chromosome initial, il est créé pour évaluer la "qualité" des solutions potentielles (les portefeuilles)
// générées par l'algorithme génétique.
// La méthode Evaluate de cette classe calcule le retour total et le risque du portefeuille, 
//et essaie de maximiser le retour tout en minimisant le risque.
var chromosome = new PortfolioChromosome(assets);

// Une population initiale de solutions potentielles est créée. 
//La population est composée d'un minimum de 50 chromosomes (solutions) et peut aller jusqu'à 100 chromosomes
var population = new Population(50, 100, chromosome);

//Un objet GeneticAlgorithm est créé en utilisant la population, la fonction de fitness, et les mécanismes de sélection, croisement, et mutation.

//RouletteWheelSelection: Sélectionne les parents proportionnellement à leur fitness.
//UniformCrossover: Combine les gènes des parents pour créer un enfant.
//UniformMutation: Change aléatoirement les gènes des chromosomes pour introduire de la diversité.
//L'algorithme génétique s'exécute pour un maximum de 100 générations (GenerationNumberTermination(100)), après quoi il s'arrête.
var ga = new GeneticAlgorithm(population, fitness, new RouletteWheelSelection(), new UniformCrossover(), new UniformMutation());
ga.Termination = new GenerationNumberTermination(100);
ga.Start();

// Affiche les résultats
var bestChromosome = ga.BestChromosome as PortfolioChromosome;
var bestWeights = bestChromosome.GetWeights();
Console.WriteLine("Meilleure allocation d'actifs trouvée :");
foreach (var asset in bestWeights.Keys)
{
    Console.WriteLine($"{asset}: {bestWeights[asset]:P2}");
}
Console.WriteLine($"Retour attendu: {portfolio.CalculateReturn(bestWeights):P2}");
Console.WriteLine($"Risque: {portfolio.CalculateRisk(bestWeights):P2}");
